In [19]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [20]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [21]:
## 전체 코드
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [22]:
## 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("stock_report/[HD현대]분기보고서(2024.11.14).pdf")
docs = loader.load()

In [23]:
## 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

In [24]:
## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [25]:
## 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [26]:
## 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [27]:
## 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """주어진 Question에 대하여 Context를 참고하여 답하시오.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [28]:
## 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(
        temperature=0.1,
        max_tokens=3000,
        model_name="gpt-4o-mini",
    )

In [ ]:
## 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "HD현대 주식 현재가가 75,700일 때, 기업의 순이익을 찾고, PER과 적정주가를 계산해줘"
response = chain.invoke(question)
print(response)